In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Types of Prompting


* Zero prompting
* Few shot prompting
* System prompting
* Role prompting
* Contextual prompting
* Step-back prompting
* Chain of thought
* Self consistency
* Tree of thoughts
* ReAct• ReActg

### Install the SDK

In [2]:
%pip install -U -q "google-generativeai>=0.8.3"

Note: you may need to restart the kernel to use updated packages.


In [3]:
import google.generativeai as genai
from IPython.display import HTML, Markdown, display

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

An Example of Zero-Shot Prompting :  1_1_movie_classification

In [5]:
model = genai.GenerativeModel(
    #'gemini-1.5-flash-001',
    'gemini-pro',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=5,
    ))

zero_shot_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response = model.generate_content(zero_shot_prompt)
print(response.text)

POSITIVE


An example of few-shot prompting : Parse pizza orders to JSON


In [6]:
model = genai.GenerativeModel(
    #'gemini-1.5-flash-001',
    'gemini-pro',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ))

zero_shot_prompt = """Parse a customer's pizza order into valid JSON:
 
 EXAMPLE:
 I want a small pizza with cheese, tomato sauce, and pepperoni.
 JSON Response:
 ```
 {
 "size": "small",
 "type": "normal",
 "ingredients": [["cheese", "tomato sauce", "peperoni"]]
 }
 ```
 EXAMPLE:
 Can I get a large pizza with tomato sauce, basil and mozzarella
 {
 "size": "large",
 "type": "normal",
 "ingredients": [["tomato sauce", "bazel", "mozzarella"]]
 }
 JSON Response:
 Now, I would like a large pizza, with the first half cheese and mozzarella. And the other tomato sauce, ham and pineapple.
 """

response = model.generate_content(zero_shot_prompt)
print(response.text)

{
 "size": "large",
 "type": "half and half",
 "ingredients": [["cheese", "mozzarella"], ["tomato sauce", "ham", "pineapple"]]
}


 An example of system prompting : Classify	movie	reviews	as	positive,	neutral	or	negative.


In [7]:
model = genai.GenerativeModel(
    #'gemini-1.5-flash-001',
    'gemini-pro',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=5,
    ))

zero_shot_prompt = """Classify movie reviews as positive, neutral or negative. Only 
return the label in uppercase.
Review: "Her" is a disturbing study revealing the direction 
humanity is headed if AI is allowed to keep evolving, 
unchecked. It's so disturbing I couldn't watch it.
Sentiment: """

response = model.generate_content(zero_shot_prompt)
print(response.text)

NEGATIVE


An example of system prompting with JSON format : Classify	movie	reviews	as	positive,	neutral	or	negative,	return	JSON.

In [8]:
model = genai.GenerativeModel(
    #'gemini-1.5-flash-001',
    'gemini-pro',
    generation_config=genai.GenerationConfig(
        temperature=1,
        top_k=40,
        top_p=0.8,
        max_output_tokens=1024,
    ))

zero_shot_prompt = """ Classify movie reviews as positive, neutral or negative. Return 
valid JSON:

Review: "Her" is a disturbing study revealing the direction 
humanity is headed if AI is allowed to keep evolving, 
unchecked. It's so disturbing I couldn't watch it.
Schema:
```
MOVIE:
{
"sentiment": String "POSITIVE" | "NEGATIVE" | "NEUTRAL",
"name": String
}
MOVIE REVIEWS:
{
"movie_reviews": [MOVIE] change to uppercase
}
```
JSON Response:
"""

response = model.generate_content(zero_shot_prompt)
print(response.text)

```json
{
  "movie_reviews": [
    {
      "sentiment": "NEGATIVE",
      "name": "HER"
    }
  ]
}
```


An example of role prompting : Act as travel guide and provide 3 travel suggestions

In [9]:
model = genai.GenerativeModel(
    #'gemini-1.5-flash-001',
    'gemini-pro',
    generation_config=genai.GenerationConfig(
        temperature=1,
        top_k=40,
        top_p=0.8,
        max_output_tokens=1024,
    ))

zero_shot_prompt = """I want you to act as a travel guide. I will write to you 
about my location and you will suggest 3 places to visit near 
me. In some cases, I will also give you the type of places I 
will visit.
My suggestion: "I am in Amsterdam and I want to visit 
only museums."
Travel Suggestions: """

response = model.generate_content(zero_shot_prompt)
print(response.text)

1. Rijksmuseum: Home to masterpieces by Rembrandt, Vermeer, and Hals.
2. Van Gogh Museum: Showcasing the largest collection of Van Gogh's works.
3. Anne Frank House: A poignant museum dedicated to the life of Anne Frank during World War II.


Some styles to use for effectiveness in text generation : Confrontational, Descriptive, Direct, Formal, Humorous, Influential, Informal, 
Inspirational  Persuasive

In [10]:
model = genai.GenerativeModel(
    #'gemini-1.5-flash-001',
    'gemini-pro',
    generation_config=genai.GenerationConfig(
        temperature=1,
        top_k=40,
        top_p=0.8,
        max_output_tokens=1024,
    ))

zero_shot_prompt = """I want you to act as a travel guide. I will write to you about 
my location and you will suggest 3 places to visit near me in 
a humorous style.
 My suggestion: "I am in Manhattan."
 Travel Suggestions: """

response = model.generate_content(zero_shot_prompt)
print(response.text)

Greetings from the bustling heart of Manhattan! You've landed in the epicenter of all things vibrant, where the streets hum with a symphony of sirens, honking cabs, and the cheerful chatter of tourists.

Now, let's get down to the nitty-gritty: where should you venture next?

1. **Times Square: A Digital Wonderland**
Stroll into the neon-drenched kingdom of Times Square, where towering billboards compete for your attention like digital gladiators. Watch as the lights dance and the crowds surge, creating a sensory overload that will leave you wondering if you've stepped into a futuristic fever dream.

2. **Central Park: A Verdant Oasis**
Escape the urban jungle and seek refuge in the sprawling green expanse of Central Park. Here, you can trade the cacophony of traffic for the gentle rustling of leaves and the cheerful chirping of birds. Rent a rowboat and glide across the tranquil waters of the lake, or simply bask in the sunshine and pretend you're in a nature documentary.

3. **The Em

An example of contextual prompting : Suggest	articles	for	a	blog	about	retro	games 

In [11]:
model = genai.GenerativeModel(
    #'gemini-1.5-flash-001',
    'gemini-pro',
    generation_config=genai.GenerationConfig(
        temperature=1,
        top_k=40,
        top_p=0.8,
        max_output_tokens=1024,
    ))

zero_shot_prompt = """Context: You are writing for a blog about retro 80's arcade video games.
Suggest 3 topics to write an article about with a few lines of description of what this article should contain. """

response = model.generate_content(zero_shot_prompt)
print(response.text)

**Topic 1: The Golden Age of Arcade Gaming**

* Explore the history of arcade video games in the 1980s, including the rise of iconic titles like Pac-Man, Donkey Kong, and Space Invaders.
* Discuss the technological advancements that fueled the arcade revolution and the social impact of these games.

**Topic 2: The Masters of Pixel Art**

* Showcase the legendary pixel artists behind the iconic graphics of 80s arcade games.
* Analyze the techniques and inspirations that created these unforgettable visuals, highlighting the challenges and breakthroughs in this artistic medium.
* Interview or feature the artists responsible for some of the most beloved pixel art in gaming history.

**Topic 3: The Evolution of Arcade Cabinet Design**

* Examine the evolution of arcade cabinet design from the early days of Pong to the elaborate and immersive machines of the 1980s.
* Discuss the technological innovations, ergonomic considerations, and artistic flourishes that shaped these iconic gaming platf

A traditional prompt before we compare it with a step back prompt :  Write	a	storyline	for	a	level	of	a	first-person	shooter	video	game. 

In [12]:
model = genai.GenerativeModel(
    #'gemini-1.5-flash-001',
    'gemini-pro',
    generation_config=genai.GenerationConfig(
        temperature=1,
        top_k=40,
        top_p=0.8,
        max_output_tokens=1024,
    ))

zero_shot_prompt = """Write a one paragraph storyline for a new level of a first
person shooter video game that is challenging and engaging. """

response = model.generate_content(zero_shot_prompt)
print(response.text)

In the depths of a desolate industrial complex, a lone operative embarks on a perilous mission to neutralize a rogue AI. As they navigate through labyrinthine corridors and abandoned machinery, they encounter waves of relentless enemies, from armed drones to towering mechs. Amidst the chaos, the AI's malicious voice taunts them, manipulating the environment to their disadvantage. The operative must rely on their sharp reflexes, tactical decision-making, and an arsenal of advanced weaponry to outwit their adversaries and confront the AI's core, where a final showdown awaits, testing their limits and pushing them to the brink of their abilities.


An example of prompting for self consistency

In [13]:
model = genai.GenerativeModel(
    #'gemini-1.5-flash-001',
    'gemini-pro',
    generation_config=genai.GenerationConfig(
        temperature=1,
        top_k=40,
        top_p=0.8,
        max_output_tokens=1024,
    ))

zero_shot_prompt = """Based on popular first-person shooter action games, what are 5 fictional key settings that contribute to a challenging and engaging level storyline in a first-person shooter video game? """

response = model.generate_content(zero_shot_prompt)
print(response.text)

1. **Abandoned Industrial Complex:** A desolate and crumbling factory, littered with machinery, overgrown with vegetation, and haunted by the echoes of past workers. This setting provides ample opportunities for cover-based combat, stealthy infiltrations, and tense encounters with lurking enemies.

2. **Urban Ruins:** A war-torn city, reduced to rubble and chaos. Broken buildings, shattered streets, and twisted wreckage create a labyrinthine environment that tests the player's navigation skills. Close-quarters combat, sniper battles, and desperate last stands amidst the ruins make for an intense and immersive experience.

3. **Ancient Tomb or Temple:** An underground labyrinth filled with hidden traps, enigmatic puzzles, and eerie atmosphere. Players must navigate through narrow passages, decipher ancient runes, and face guardians and monsters from forgotten eras. The setting evokes a sense of mystery, danger, and the allure of uncovering lost secrets.

4. **Space Station or Starship:*

An example of prompting for self consistency

In [14]:
model = genai.GenerativeModel(
    #'gemini-1.5-flash-001',
    'gemini-pro',
    generation_config=genai.GenerationConfig(
        temperature=1,
        top_k=40,
        top_p=0.8,
        max_output_tokens=1024,
    ))

zero_shot_prompt = """Context: 5 engaging themes for a first person shooter video game:
 1. **Abandoned Military Base**: A sprawling, post-apocalyptic 
military complex crawling with mutated soldiers and rogue 
robots, ideal for challenging firearm combat.
 2. **Cyberpunk City**: A neon-lit, futuristic urban environment 
with towering skyscrapers and dense alleyways, featuring 
cybernetically enhanced enemies and hacking mechanics.
 3. **Alien Spaceship**: A vast alien vessel stranded on 
Earth, with eerie corridors, zero-gravity sections, and 
extraterrestrial creatures to encounter.
 4. **Zombie-Infested Town**: A desolate town overrun by hordes of 
aggressive zombies, featuring intense close-quarters combat and 
puzzle-solving to find safe passage.
 5. **Underwater Research Facility**: A deep-sea laboratory flooded 
with water, filled with mutated aquatic creatures, and requiring 
stealth and underwater exploration skills to survive.
 Take one of the themes and write a one paragraph storyline 
for a new level of a first-person shooter video game that is 
challenging and engaging."""

response = model.generate_content(zero_shot_prompt)
print(response.text)

**Cyberpunk City: The Neon Underworld**

Descend into the neon-drenched underbelly of the city as a lone hacker on a perilous mission to infiltrate a secretive corporation. Navigate through labyrinthine alleyways and towering skyscrapers, dodging laser beams and avoiding detection by heavily armed guards. Encounter cybernetically enhanced enemies who possess advanced weaponry and hacking abilities. Utilize stealth and hacking skills to disable security systems, sabotage defenses, and access restricted areas. As you delve deeper into the corporation's network, uncover a sinister plot that threatens the city's very existence. Engage in intense firefights against formidable cyborgs and navigate treacherous zero-gravity environments. The neon lights flicker and cast an eerie glow as you fight to expose the truth and save the city from impending doom.


An example of a prompt which is trying to solve a mathematical problem

In [15]:
#prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
#am 20 years old. How old is my partner? Return the answer immediately."""

prompt = """When I was 3 years old, my partner was 3 times my age. Now, I 
am 20 years old. How old is my partner?"""

model = genai.GenerativeModel('gemini-pro')
response = model.generate_content(prompt)

print(response.text)

29 years old


An example of Chain of Thought prompting

In [16]:
#prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
#am 20 years old. How old is my partner? Return the answer immediately."""

prompt = """When I was 3 years old, my partner was 3 times my age. Now, 
I am 20 years old. How old is my partner? Let's think step 
by step."""

model = genai.GenerativeModel('gemini-pro')
response = model.generate_content(prompt)

print(response.text)

**Step 1: Find my partner's age when I was 3 years old.**

When I was 3, my partner was 3 times my age. So, if I was x years old when I was 3, my partner was 3x years old then.

**Step 2: Find my partner's current age.**

Now, I am 20 years old. So, my partner is currently 20 + x years old.

**Step 3: Substitute my age when I was 3.**

We know that when I was 3, my partner was 3 times my age. So, x = 3. Substituting this into the expression for my partner's current age, we get:

My partner's current age = 20 + x = 20 + 3 = **23 years old**


 An example of chain of thought prompting with a single-shot

In [17]:
#prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
#am 20 years old. How old is my partner? Return the answer immediately."""

prompt = """Q: When my brother was 2 years old, I was double his age. Now 
I am 40 years old. How old is my brother? Let's think step 
by step.
A: When my brother was 2 years, I was 2 * 2 = 4 years old. 
That's an age difference of 2 years and I am older. Now I am 40 
years old, so my brother is 40 - 2  = 38 years old. The answer 
is 38.
Q: When I was 3 years old, my partner was 3 times my age. Now, 
I am 20 years old. How old is my partner? Let's think step 
by step.
A:"""

model = genai.GenerativeModel('gemini-pro')
response = model.generate_content(prompt)

print(response.text)

When you were 3 years old, your partner was 3 * 3 = 9 
years old. That's an age difference of 6 years and your partner 
is older. Now you are 20 years old, so your partner is 20 + 6  
= 26 years old. The answer is 26.


An example of prompting for self consistency

Attempt 1

In [18]:
#prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
#am 20 years old. How old is my partner? Return the answer immediately."""

prompt = """EMAIL:
```
Hi,
I have seen you use Wordpress for your website. A great open 
source content management system. I have used it in the past 
too. It comes with lots of great user plugins. And it's pretty 
easy to set up.
I did notice a bug in the contact form, which happens when 
you select the name field. See the attached screenshot of me 
entering text in the name field. Notice the JavaScript alert 
box that I inv0k3d.
But for the rest it's a great website. I enjoy reading it. Feel 
free to leave the bug in the website, because it gives me more 
interesting things to read.

Cheers,
Harry the Hacker.
```
Classify the above email as IMPORTANT or NOT IMPORTANT. Let's think step by step and explain why.
"""

model = genai.GenerativeModel('gemini-pro',
                              generation_config=genai.GenerationConfig(temperature=1, top_k=40, top_p=0.8, max_output_tokens=1024)
                             )
response = model.generate_content(prompt)

print(response.text)

**IMPORTANT**

**Step 1: Identify the purpose of the email.**

The purpose of the email is to inform the recipient of a bug in the contact form of their website.

**Step 2: Assess the severity of the bug.**

The bug involves a JavaScript alert box that appears when the name field is selected. This could be a security issue, as it could allow an attacker to inject malicious code into the website.

**Step 3: Consider the potential impact of the bug.**

If the bug is not fixed, it could allow an attacker to gain access to the website's data or to compromise the website's security.

**Step 4: Evaluate the credibility of the sender.**

The sender identifies himself as "Harry the Hacker," which suggests that he is knowledgeable about website security and that he is likely to be credible.

**Conclusion:**

Based on the above factors, the email is classified as **IMPORTANT**. The bug that is reported could have a significant impact on the website's security, and the sender is credible. The rec

Attempt 2

In [19]:
#prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
#am 20 years old. How old is my partner? Return the answer immediately."""

prompt = """EMAIL:
```
Hi,
I have seen you use Wordpress for your website. A great open 
source content management system. I have used it in the past 
too. It comes with lots of great user plugins. And it's pretty 
easy to set up.
I did notice a bug in the contact form, which happens when 
you select the name field. See the attached screenshot of me 
entering text in the name field. Notice the JavaScript alert 
box that I inv0k3d.
But for the rest it's a great website. I enjoy reading it. Feel 
free to leave the bug in the website, because it gives me more 
interesting things to read.

Cheers,
Harry the Hacker.
```
Classify the above email as IMPORTANT or NOT IMPORTANT. Let's think step by step and explain why.
"""

model = genai.GenerativeModel('gemini-pro',
                              generation_config=genai.GenerationConfig(temperature=1, top_k=40, top_p=0.8, max_output_tokens=1024)
                             )
response = model.generate_content(prompt)

print(response.text)

**IMPORTANT**

**Explanation:**

The email:

* Reports a security vulnerability (bug) in the contact form.
* Includes a screenshot of the vulnerability being exploited.
* States that the bug is currently exploitable.
* Indicates that the sender has knowledge of web security and hacking techniques.

These factors indicate that the email is important and should be addressed promptly to prevent potential security breaches.


Attempt 3

In [20]:
#prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
#am 20 years old. How old is my partner? Return the answer immediately."""

prompt = """EMAIL:
```
Hi,
I have seen you use Wordpress for your website. A great open 
source content management system. I have used it in the past 
too. It comes with lots of great user plugins. And it's pretty 
easy to set up.
I did notice a bug in the contact form, which happens when 
you select the name field. See the attached screenshot of me 
entering text in the name field. Notice the JavaScript alert 
box that I inv0k3d.
But for the rest it's a great website. I enjoy reading it. Feel 
free to leave the bug in the website, because it gives me more 
interesting things to read.

Cheers,
Harry the Hacker.
```
Classify the above email as IMPORTANT or NOT IMPORTANT. Let's think step by step and explain why.
"""

model = genai.GenerativeModel('gemini-pro',
                              generation_config=genai.GenerationConfig(temperature=1, top_k=40, top_p=0.8, max_output_tokens=1024)
                             )
response = model.generate_content(prompt)

print(response.text)

**IMPORTANT**

**Explanation:**

* The email mentions a specific bug in the contact form, which could potentially compromise the security of the website.
* The email includes a screenshot as evidence of the bug.
* The sender claims to have used WordPress in the past, which indicates some level of technical knowledge.
* The sender's email address could be used to identify the attacker.
* The sender's name, "Harry the Hacker," suggests that they may have malicious intent.
* The email's tone is professional and polite, which could be an attempt to deceive the recipient.


Creating a ReAct Agent with LangChain and VertexAI

In [21]:
#%pip install --user langchain==0.0.310 \
#                    google-cloud-aiplatform==1.35.0 \
#                    prettyprinter==0.18.0 \
#                    wikipedia==1.4.0 \
#                    chromadb==0.3.26 \
#                    tiktoken==0.5.1 \
#                    tabulate==0.9.0 \
#                    sqlalchemy-bigquery==1.8.0 \
#                    google-cloud-bigquery==3.11.4

In [22]:
#%pip install -U -q langchain

In [23]:
#%pip install -U langchain-community

In [24]:
'''
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)
'''

'\n# Restart kernel after installs so that your environment can access the new packages\nimport IPython\n\napp = IPython.Application.instance()\napp.kernel.do_shutdown(True)\n'

**Authenticate your notebook environment (Colab only)***


In [25]:
'''
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()
    print("Authenticated")
'''    

'\nimport sys\n\nif "google.colab" in sys.modules:\n    from google.colab import auth\n\n    auth.authenticate_user()\n    print("Authenticated")\n'

**Import packages**

In [26]:
'''
import IPython
from IPython.display import Markdown, display
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import VertexAI

PROJECT_ID = "your-project-here"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-pro"  # @param {type:"string"}

prompt = "How many kids do that band members of Metallica have?"

llm = VertexAI(temperature=0.1)
tools = load_tools(["serpai"], llm=llm)

agent=initialize_agent(tools, llm,
                      agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.run(prompt)
'''

'\nimport IPython\nfrom IPython.display import Markdown, display\nfrom langchain.agents import load_tools\nfrom langchain.agents import initialize_agent\nfrom langchain.agents import AgentType\nfrom langchain.llms import VertexAI\n\nPROJECT_ID = "your-project-here"  # @param {type:"string"}\nLOCATION = "us-central1"  # @param {type:"string"}\nMODEL_NAME = "gemini-pro"  # @param {type:"string"}\n\nprompt = "How many kids do that band members of Metallica have?"\n\nllm = VertexAI(temperature=0.1)\ntools = load_tools(["serpai"], llm=llm)\n\nagent=initialize_agent(tools, llm,\n                      agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)\n\nagent.run(prompt)\n'

Using Gemini to program code : Write	a	prompt	to	write	code	in	Bash	to	rename	files	in	a	folder.


In [27]:
#prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
#am 20 years old. How old is my partner? Return the answer immediately."""

prompt = """
Write a code snippet in Bash, which asks for a folder name. 
Then it takes the contents of the folder and renames all the 
files inside by prepending the name draft to the file name.
"""

model = genai.GenerativeModel('gemini-pro',
                              generation_config=genai.GenerationConfig(temperature=0.1, top_p=1, max_output_tokens=1024)
                             )
response = model.generate_content(prompt)

print(response.text)

```bash
#!/bin/bash

# Ask for a folder name
echo "Enter the folder name:"
read foldername

# Check if the folder exists
if [ ! -d "$foldername" ]; then
  echo "Folder $foldername does not exist."
  exit 1
fi

# Get the contents of the folder
files=$(ls "$foldername")

# Rename all the files
for file in $files; do
  mv "$foldername/$file" "$foldername/draft-$file"
done

echo "Files renamed successfully."
```


A prompt for explaining code : Write a prompt to explain Bash code

In [28]:
#prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
#am 20 years old. How old is my partner? Return the answer immediately."""

prompt = """
 Explain to me the below Bash code:
 ```
 #!/bin/bash
 echo "Enter the folder name: "
 read folder_name
 if [ ! -d "$folder_name" ]; then
 echo "Folder does not exist."
 exit 1
 fi
 files=( "$folder_name"/* )
 for file in "${files[@]}"; do
 new_file_name="draft_$(basename "$file")"
 mv "$file" "$new_file_name"
 done
 echo "Files renamed successfully."
 ```
"""

model = genai.GenerativeModel('gemini-pro',
                              generation_config=genai.GenerationConfig(temperature=0.1, top_p=1, max_output_tokens=1024)
                             )
response = model.generate_content(prompt)

print(response.text)

This Bash code performs the following tasks:

1. **Prompt for Folder Name**: It prompts the user to enter the name of a folder.

2. **Check Folder Existence**: It checks if the specified folder exists using the `[ ! -d "$folder_name" ]` condition. If the folder does not exist, it prints "Folder does not exist." and exits the script with an error code of 1.

3. **Get Files in Folder**: It creates an array `files` containing the paths to all files in the specified folder using the `files=( "$folder_name"/* )` command.

4. **Rename Files**: It iterates through the files in the `files` array using a `for` loop. For each file, it generates a new file name with the prefix "draft_" followed by the original file name using the `new_file_name="draft_$(basename "$file")"` command. It then renames the file using the `mv "$file" "$new_file_name"` command.

5. **Success Message**: After renaming all files, it prints "Files renamed successfully." to indicate the completion of the operation.


A prompt for translating code from Bash to Python :  Write a prompt to translate Bash code to Python

In [29]:
#prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
#am 20 years old. How old is my partner? Return the answer immediately."""

prompt = """
 Translate the below Bash code to a Python snippet.
 ```bash
 #!/bin/bash
 echo "Enter the folder name: "
 read folder_name
 if [ ! -d "$folder_name" ]; then
 echo "Folder does not exist."
 exit 1
 fi
 files=( "$folder_name"/* )
 for file in "${files[@]}"; do
 new_file_name="draft_$(basename "$file")"
 mv "$file" "$new_file_name"
 done
 echo "Files renamed successfully."
 ```
"""

model = genai.GenerativeModel('gemini-pro',
                              generation_config=genai.GenerationConfig(temperature=0.1, top_p=1, max_output_tokens=1024)
                             )
response = model.generate_content(prompt)

print(response.text)

```python
import os

# Prompt the user to enter the folder name
folder_name = input("Enter the folder name: ")

# Check if the folder exists
if not os.path.isdir(folder_name):
    print("Folder does not exist.")
    exit(1)

# Get a list of all files in the folder
files = os.listdir(folder_name)

# Rename each file with a "draft_" prefix
for file in files:
    new_file_name = "draft_" + os.path.basename(file)
    os.rename(os.path.join(folder_name, file), os.path.join(folder_name, new_file_name))

# Print a success message
print("Files renamed successfully.")
```


 A prompt for debugging and reviewing Python code

In [30]:
#prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
#am 20 years old. How old is my partner? Return the answer immediately."""

prompt = """
The below Python code gives an error:
 Traceback (most recent call last):
                                                   File "/
 Users/leeboonstra/Documents/test_folder/rename_files.py", line 
7, in <module>
    text = toUpperCase(prefix)
 NameError: name 'toUpperCase' is not defined
 Debug what's wrong and explain how I can improve the code.
 ```python
 import os
 import shutil
 # Get the folder name from the user
 folder_name = input("Enter the folder name: ")
 prefix = input("Enter the string to prepend to the filename: ")
 text = toUpperCase(prefix)
 # Check if the folder exists
 if not os.path.isdir(folder_name):
    print("Folder does not exist.")
    e x it(1)
 # Get a list of all files in the folder
 files = os.listdir(folder_name)
 # Iterate over the files
 for file in files:
 # Create a new file name with the prefix "draft_"
   new_filename = f"{text}_{file}"
 # Move the file to the new name
 shutil.move(os.path.join(folder_name, file), 
 os.path.join(folder_name, new_file_name))
 # Print a success message
 print("Files renamed successfully.")
 ```
"""

model = genai.GenerativeModel('gemini-pro',
                              generation_config=genai.GenerationConfig(temperature=0.1, top_p=1, max_output_tokens=1024)
                             )
response = model.generate_content(prompt)

print(response.text)

The error is caused by the use of the undefined function `toUpperCase`. To fix this, you can use the built-in `upper` function to convert the prefix to uppercase. Here's the corrected code:

```python
import os
import shutil

# Get the folder name from the user
folder_name = input("Enter the folder name: ")
prefix = input("Enter the string to prepend to the filename: ")
text = prefix.upper()

# Check if the folder exists
if not os.path.isdir(folder_name):
    print("Folder does not exist.")
    exit(1)

# Get a list of all files in the folder
files = os.listdir(folder_name)

# Iterate over the files
for file in files:
    # Create a new file name with the prefix "draft_"
    new_filename = f"{text}_{file}"
    # Move the file to the new name
    shutil.move(os.path.join(folder_name, file),
                os.path.join(folder_name, new_filename))

# Print a success message
print("Files renamed successfully.")
```

Here are some additional improvements you can make to the code:

* Use a 